In [ ]:
import zipfile
import xml.etree.ElementTree as ET
import requests
import io
import pandas as pd
API_KEY = ''

In [2]:


URL = f'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}'

# 1. 다운로드
res = requests.get(URL)
with zipfile.ZipFile(io.BytesIO(res.content)) as z:
    with z.open('CORPCODE.xml') as xml_file:
        # 2. 빠르게 파싱
        data = []
        for event, elem in ET.iterparse(xml_file, events=('end',)):
            if elem.tag == 'list':
                stock_code = elem.find('stock_code').text
                if stock_code:  # 상장기업만
                    data.append({
                        'corp_code': elem.find('corp_code').text,
                        'corp_name': elem.find('corp_name').text,
                        'stock_code': stock_code,
                        'modify_date': elem.find('modify_date').text
                    })
                elem.clear()

# 3. 저장
df = pd.DataFrame(data)
df.to_csv('상장기업_corp_code.csv', index=False)

In [8]:
# 1. CSV 불러오기
df = pd.read_csv('상장기업_corp_code.csv')

# 2. stock_code가 비어있는 행 제거 (NaN 또는 빈 문자열)
df = df[df['stock_code'].notna()]  # NaN 제거
df = df[df['stock_code'].astype(str).str.strip() != '']  # 공백 제거

# 3. 결과 확인 (선택)
print(f"남은 행 수: {len(df)}")

# 4. 저장
df.to_csv('상장기업_corp_code_필터링.csv', index=False)

남은 행 수: 3853


In [20]:
# 상장폐지 관련 공시 검색
url = f'http://opendart.fss.or.kr/api/search.json?crtfc_key={API_KEY}&bgn_de=20200101&end_de=20231231&report_nm=상장폐지'
response = requests.get(url)
data = response.json()
print(data)

{'message': '잘못된 URL입니다. 확인해주세요(/api/search.json)', 'status': '101'}


In [ ]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('stock_corp_code.csv')

# 상장폐지된 기업의 stock_code 리스트
delisted_stock_codes = ['005930', '000660', '035420', ...]  # 상장폐지된 주식 코드들

# 상장폐지된 기업 제외하기
df_cleaned = df[~df['stock_code'].isin(delisted_stock_codes)]

# 결과를 새로운 CSV 파일로 저장
df_cleaned.to_csv('cleaned_file.csv', index=False)